In [31]:
!pip install prometheus_api_client > /dev/null 2>&1
import os
import json
import hashlib
import pandas as pd
from datetime import datetime, timezone
import requests
from IPython import get_ipython
from IPython.core.interactiveshell import ExecutionResult
from prometheus_api_client import PrometheusConnect

class ExperimentTracker:
    def __init__(self):
        self.experiment_id = self._new_experiment_id()
        self.start_time = datetime.now(timezone.utc)  # Timezone-aware
        self.executed_cells = 0
        self.failed_cells = 0
        self.logs = []
        self.log_dir = f"/home/jovyan/experiment_logs/{self.experiment_id}/output/logs"
        self.metrics_dir = f"/home/jovyan/experiment_logs/{self.experiment_id}/output/metrics"
        os.makedirs(self.log_dir, exist_ok=True)
        os.makedirs(self.metrics_dir, exist_ok=True)
        print(f"[Experiment Tracking] Started experiment {self.experiment_id}")

    def export_scaph_metrics(self, start_time, end_time):
        prom_url = "https://mc-a4.lab.uvalight.net/prometheus-goncalo/"
        prom = PrometheusConnect(url=prom_url, disable_ssl=True)
        try:
            all_metrics = prom.get_label_values(label_name="__name__")
            scaph_metrics = sorted([m for m in all_metrics if m.startswith("scaph_")])
            print(f"[Experiment Tracking] Detected Scaphandre metrics: {scaph_metrics}")
        except Exception as e:
            print(f"[Experiment Tracking] Failed to fetch Prometheus metrics: {e}")
            return

        all_data = {}
        for metric in scaph_metrics:
            print(f"[Experiment Tracking] Exporting {metric}")
            try:
                data = prom.get_metric_range_data(metric_name=metric, start_time=start_time, end_time=end_time)
                if not data or "values" not in data[0]:
                    print(f"[Experiment Tracking] No data for {metric}")
                    continue
                for ts, val in data[0]["values"]:
                    dt = datetime.fromtimestamp(float(ts), tz=timezone.utc)  # Timezone-aware
                    if dt not in all_data:
                        all_data[dt] = {}
                    all_data[dt][metric] = float(val)
            except Exception as e:
                print(f"[Experiment Tracking] Error fetching data for {metric}: {e}")

        if not all_data:
            print(f"[Experiment Tracking] No Scaphandre metrics collected.")
            return

        rows = [{"timestamp": dt, **metrics} for dt, metrics in sorted(all_data.items())]
        df = pd.DataFrame(rows).sort_values("timestamp")
        csv_path = os.path.join(self.metrics_dir, "scaph_metrics_combined.csv")
        df.to_csv(csv_path, index=False)
        print(f"[Experiment Tracking] Wrote {len(df)} rows to {csv_path}")

    def _new_experiment_id(self):
        ts = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        return f"exp-{hashlib.sha256(ts.encode()).hexdigest()[:8]}-{ts}"

    def pre_run_cell(self, info):
        self.cell_start_time = datetime.now(timezone.utc)  # Timezone-aware
        self.current_cell_code = info.raw_cell.strip()
        self.logs.append(f"[{self.cell_start_time}] Starting cell execution.")

    def post_run_cell(self, result: ExecutionResult):
        cell_end_time = datetime.now(timezone.utc)  # Timezone-aware
        if not hasattr(self, "cell_start_time"):
            self.cell_start_time = self.start_time
            self.current_cell_code = "<unknown cell>"
            self.logs.append(f"[{cell_end_time}] WARNING: pre_run_cell hook may not have run.")
        duration = (cell_end_time - self.cell_start_time).total_seconds()
        self.executed_cells += 1
        cell_summary = (self.current_cell_code[:60] + '...') if len(self.current_cell_code) > 60 else self.current_cell_code
        if result.error_in_exec:
            self.failed_cells += 1
            error_msg = f"[{cell_end_time}] ERROR in cell ({cell_summary}): {repr(result.error_in_exec)}"
            self.logs.append(error_msg)
            print(error_msg)
        else:
            success_msg = f"[{cell_end_time}] SUCCESS in cell ({cell_summary}) [{duration:.2f}s]"
            self.logs.append(success_msg)
            print(success_msg)

    def end_experiment(self):
        end_time = datetime.now(timezone.utc)  # Timezone-aware
        duration = (end_time - self.start_time).total_seconds()
        summary = {
            "experiment_id": self.experiment_id,
            "start_time": str(self.start_time),
            "end_time": str(end_time),
            "duration_sec": duration,
            "cells_executed": self.executed_cells,
            "cells_failed": self.failed_cells
        }
        summary_path = f"/home/jovyan/experiment_logs/{self.experiment_id}/metadata.json"
        with open(summary_path, "w") as f:
            json.dump(summary, f, indent=2)
        log_path = os.path.join(self.log_dir, "log.txt")
        with open(log_path, "w") as f:
            f.write("\n".join(self.logs))
        print(f"[Experiment Tracking] Experiment {self.experiment_id} completed and logged.")
        self.export_scaph_metrics(self.start_time, end_time)

# Register hooks.
tracker = ExperimentTracker()
ip = get_ipython()
ip.events.register('pre_run_cell', tracker.pre_run_cell)
ip.events.register('post_run_cell', tracker.post_run_cell)
print("[Experiment Tracking] IPython hooks registered.")


[Experiment Tracking] Started experiment exp-b2ea4342-20250602055601
[Experiment Tracking] IPython hooks registered.
[2025-06-02 05:56:01.669306] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669400+00:00] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669456+00:00] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669495+00:00] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669536+00:00] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669569+00:00] SUCCESS in cell (!pip install prometheus_api_client > /dev/null 2>&1
import o...) [2.13s]
[2025-06-02 05:56:01.669607+00:00] SUCCESS in cell (<unknown cell>) [0.00s]


# Tutorial EcoJupyter Tool
This tutorial helps you kickstart the use of the [EcoJupyter](https://github.com/g-uva/EcoJupyter) and [JupyterK8sTool](https://github.com/g-uva/JupyterK8sMonitor) tools.

## Before we start
Before we start, you must run from the menu: `Run > Run All Cells`. Why? This Notebook contains some nice UI that is only accessible after the cells are executed.

## Step 1: install `EcoJupyter`
First, let's install the tool using the `pip` command! 👇

In [26]:
!pip install --upgrade ecojupyter

[2025-06-02 05:46:53.663598] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]
[2025-06-02 05:46:53.663716+00:00] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]
[2025-06-02 05:46:53.663757+00:00] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]
[2025-06-02 05:46:53.663786+00:00] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]
[2025-06-02 05:46:53.663819+00:00] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]
[2025-06-02 05:46:53.663854+00:00] SUCCESS in cell (!pip install --upgrade ecojupyter) [4.07s]


If you see the message `Successfully installed ecojupyter-<version>`, congratulations! `EcoJupyter` has been successfully installed in this notebook. 👏

You can open your newly installed plugin by:
1. Refreshing the page.
2. Click on `View > Activate Command Palette`.
3. Look for `Open GreenDIGIT Dashboard`.

## Step 2: Install monitoring agent `JupyterK8sMonitor`
You must run this script in order to download and run the tool that will expose the metrics that will later be read by `EcoJupyter` tool.


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os

output = widgets.Output()
spinner = widgets.HTML("<i>Idle</i>")

def run_step(description, command, shell=False, env=None, cwd=None):
    print(f"\n---- {description} ----")
    try:
        result = subprocess.run(
            command,
            shell=shell,
            check=True,
            capture_output=True,
            text=True,
            env=env,
            cwd=cwd
        )
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"❌ ERROR during: {description}")
        print(f"Output:\n{e.output}")
        print(f"Stderr:\n{e.stderr}")
        raise

def on_scaphandre_click(b):
    with output:
        output.clear_output()
        try:
            spinner.value = "<i class='fa fa-spinner fa-spin'></i> Installing Scaphandre + Prometheus..."
            
            # Ensure bin directory exists
            bin_dir = "/home/jovyan/.bin"
            os.makedirs(bin_dir, exist_ok=True)
            print(f"Ensured {bin_dir} exists.")
            
            # Download the script
            run_step("Downloading installation script", [
                "curl", "-L", "-o", os.path.join(bin_dir, "install-scaphandre-prometheus.sh"),
                "https://raw.githubusercontent.com/g-uva/jupyterhub-scaphandre-monitor/refs/heads/master/scaphandre-prometheus-ownpod/install-scaphandre-prometheus.sh"
            ])
            
            # Make it executable
            run_step("Setting script permissions", [
                "chmod", "+x", os.path.join(bin_dir, "install-scaphandre-prometheus.sh")
            ])
            
            # Run the script with shell=True and modify it to background tasks using nohup &
            run_step("Running installation script with background tasks", f"bash -c 'nohup {os.path.join(bin_dir, 'install-scaphandre-prometheus.sh')} > {os.path.join(bin_dir, 'install.log')} 2>&1 &'", shell=True)
            
            spinner.value = "<span style='color:green;'>✔️ Completed successfully!</span>"
            print("✅ Scaphandre + Prometheus installation started in background.")
        
        except Exception as e:
            spinner.value = f"<span style='color:red;'>❌ Installation failed: {e}</span>"
            print(f"⚠️ Installation failed: {e}")


[2025-06-02 05:46:53.695400] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]
[2025-06-02 05:46:53.695702+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]
[2025-06-02 05:46:53.695736+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]
[2025-06-02 05:46:53.695755+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]
[2025-06-02 05:46:53.695774+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]
[2025-06-02 05:46:53.695792+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.02s]


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import threading
import os
import requests

spinner = widgets.HTML("<i>Idle</i>")
output = widgets.Output()

def run_step(description, command, shell=False, env=None):
    spinner.value = f"<i class='fa fa-spinner fa-spin'></i> {description}"
    subprocess.run(
        command,
        shell=shell,
        check=True,
        capture_output=True,
        text=True,
        env=env
    )

def check_scaphandre_and_prometheus():
    with output:
        output.clear_output()
        print("Checking Scaphandre and Prometheus installations...")
        
        scaphandre_path = "/usr/local/bin/scaphandre"
        prometheus_dir = "/home/jovyan/.bin/prometheus-unzipped"
        prometheus_config = "/home/jovyan/.bin/prometheus.yml"
        
        scaphandre_ok = os.path.isfile(scaphandre_path)
        prometheus_ok = os.path.isdir(prometheus_dir) and os.path.isfile(prometheus_config)
        
        if scaphandre_ok and prometheus_ok:
            print("✅ Scaphandre and Prometheus are already installed.")
        else:
            if not scaphandre_ok:
                print("❌ Scaphandre binary not found at", scaphandre_path)
            if not prometheus_ok:
                print(f"❌ Prometheus folder or config missing:\n- Folder: {prometheus_dir}\n- Config: {prometheus_config}")

def check_prometheus_running():
    with output:
        print("\nChecking if Prometheus server is running on localhost:9090...")
        try:
            response = requests.get("http://localhost:9090")
            if response.status_code == 200:
                print("✅ Prometheus server is already running.")
            else:
                print(f"⚠️ Prometheus server responded with status code {response.status_code}.")
        except requests.ConnectionError:
            print("❌ Prometheus server is NOT running on localhost:9090.")

def run_installation():
    try:
        home = os.path.expanduser("~")
        env = os.environ.copy()
        env["HOME"] = home

        run_step("APT update", ["sudo", "apt-get", "update"])
        run_step("Install dependencies", ["sudo", "apt-get", "install", "-y", "pkg-config", "libssl-dev", "lsof"])

        run_step(
            "Install Rust and build Scaphandre",
            """
            bash -c '
            curl --proto "=https" --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
            source $HOME/.cargo/env
            rustup install 1.65.0
            rustup override set 1.65.0
            git clone https://github.com/hubblo-org/scaphandre.git
            cd scaphandre
            cargo build --release
            sudo mv ./target/release/scaphandre /usr/local/bin/
            cd ~ && rm -rf scaphandre
            '
            """,
            shell=True
        )

        run_step("Start Scaphandre", "nohup scaphandre prometheus --address=0.0.0.0 --port=8081 --containers > /home/jovyan/.bin/scaphandre.log 2>&1 &", shell=True)

        run_step("Remove old Prometheus", ["sudo", "rm", "-rf", "/home/jovyan/.bin/prometheus-unzipped"])
        run_step("Download Prometheus", ["wget", "https://github.com/prometheus/prometheus/releases/download/v2.52.0/prometheus-2.52.0.linux-amd64.tar.gz", "-P", "/home/jovyan/.bin/"])
        run_step("Unzip Prometheus", ["tar", "xzf", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64.tar.gz", "-C", "/home/jovyan/.bin/"])
        run_step("Move Prometheus", ["mv", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64", "/home/jovyan/.bin/prometheus-unzipped"])
        run_step("Cleanup Prometheus archive", ["rm", "-rf", "/home/jovyan/.bin/prometheus-2.52.0.linux-amd64.tar.gz"])

        prometheus_config = """
global:
  scrape_interval: 5s

scrape_configs:
  - job_name: 'scaphandre-local'
    static_configs:
      - targets: ['localhost:8081']
"""
        config_path = "/home/jovyan/.bin/prometheus.yml"
        with open(config_path, "w") as f:
            f.write(prometheus_config)

        spinner.value = "<span style='color:green;'>✔️ Completed successfully!</span>"

    except Exception as e:
        spinner.value = f"<span style='color:red;'>❌ Installation failed: {e}</span>"

def start_prometheus():
    try:
        run_step("Start Prometheus", "nohup /home/jovyan/.bin/prometheus-unzipped/prometheus --config.file=/home/jovyan/.bin/prometheus.yml --web.listen-address=0.0.0.0:9090 > /home/jovyan/.bin/prometheus.log 2>&1 &", shell=True)
        spinner.value = "<span style='color:green;'>✔️ Prometheus started successfully!</span>"
    except Exception as e:
        spinner.value = f"<span style='color:red;'>❌ Failed to start Prometheus: {e}</span>"

def on_install_click(b):
    threading.Thread(target=run_installation).start()

def on_prometheus_click(b):
    threading.Thread(target=start_prometheus).start()

def on_check_click(b):
    check_scaphandre_and_prometheus()
    check_prometheus_running()

# UI Buttons
button_scaphandre = widgets.Button(description="Install Scaphandre", layout=widgets.Layout(width='200px'))
install_button = widgets.Button(description="Install Prometheus", layout=widgets.Layout(width='200px'))
prometheus_button = widgets.Button(description="Start Prometheus", layout=widgets.Layout(width='200px'))
check_button = widgets.Button(description="Check Status", layout=widgets.Layout(width='200px'))

button_scaphandre.on_click(on_scaphandre_click)
install_button.on_click(on_install_click)
prometheus_button.on_click(on_prometheus_click)
check_button.on_click(on_check_click)

display(widgets.VBox([
    widgets.HBox([button_scaphandre, install_button, prometheus_button, check_button]),
    spinner, output
]))


[2025-06-02 05:46:53.833744] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]
[2025-06-02 05:46:53.833951+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]
[2025-06-02 05:46:53.833982+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]
[2025-06-02 05:46:53.834005+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]
[2025-06-02 05:46:53.834023+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]
[2025-06-02 05:46:53.834042+00:00] SUCCESS in cell (import ipywidgets as widgets
from IPython.display import dis...) [0.11s]


## Step 3: Run a Workload
If you do **not** have the `EcoJupyter` plugin installed and opened on the side, please follow the last instructions of Step 1.

If you do, then you're good to go! Let's spin up some stress test work by running the below cell.

> Note: if you do not see charts with data on the plugin, consider reloading the window. Sometimes it might take some time to synchronise.

In [29]:
import multiprocessing
import threading
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

cancel_event = threading.Event()
stress_running = threading.Event()

def cpu_stress(cores, duration, output_box, progress_cpu):
    def cpu_load():
        while not stop_event.is_set() and not cancel_event.is_set():
            pass
    stop_event = threading.Event()
    processes = []
    try:
        output_box.append_stdout(f"Starting {cores} CPU stress processes...\n")
        for _ in range(cores):
            p = multiprocessing.Process(target=cpu_load)
            p.start()
            processes.append(p)
        for i in range(duration):
            if cancel_event.is_set():
                output_box.append_stdout("CPU stress canceled.\n")
                break
            time.sleep(1)
            progress_cpu.value = int((i + 1) / duration * 100)
    finally:
        stop_event.set()
        for p in processes:
            p.terminate()
        progress_cpu.value = 100
        output_box.append_stdout("CPU stress test ended.\n\n")

def ram_stress(mb, duration, output_box, progress_ram):
    big_data = []
    try:
        output_box.append_stdout(f"Allocating ~{mb} MB RAM...\n")
        for _ in range(mb):
            if cancel_event.is_set():
                output_box.append_stdout("RAM allocation canceled during allocation.\n")
                return
            big_data.append(bytearray(1024 * 1024))
        for i in range(duration):
            if cancel_event.is_set():
                output_box.append_stdout("RAM stress canceled.\n")
                break
            time.sleep(1)
            progress_ram.value = int((i + 1) / duration * 100)
    except MemoryError:
        output_box.append_stdout("MemoryError: Could not allocate requested memory.\n")
    finally:
        del big_data
        progress_ram.value = 100
        output_box.append_stdout("RAM stress test ended.\n\n")

def combined_stress(cores, mb, duration, output_box, progress_cpu, progress_ram):
    output_box.append_stdout("Running combined CPU and RAM stress...\n")
    cpu_thread = threading.Thread(target=cpu_stress, args=(cores, duration, output_box, progress_cpu))
    ram_thread = threading.Thread(target=ram_stress, args=(mb, duration, output_box, progress_ram))
    cpu_thread.start()
    ram_thread.start()
    cpu_thread.join()
    ram_thread.join()
    output_box.append_stdout("Combined stress test complete.\n\n")

# Widgets
warning = widgets.HTML("<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—"
                      "setting too high may crash your environment or cause instability.")

cpu_count = multiprocessing.cpu_count()
cpu_input = widgets.BoundedIntText(value=min(2, cpu_count), min=1, max=cpu_count, layout=widgets.Layout(width='100px'))
ram_input = widgets.BoundedIntText(value=500, min=10, max=20000, step=10, layout=widgets.Layout(width='100px'))
dur_input = widgets.BoundedIntText(value=30, min=5, max=600, step=5, description="Duration (s):", layout=widgets.Layout(width='150px'))

cpu_button = widgets.Button(description="Run CPU Stress Test", button_style="warning", layout=widgets.Layout(width='150px'))
ram_button = widgets.Button(description="Run RAM Stress Test", button_style="warning", layout=widgets.Layout(width='150px'))
both_button = widgets.Button(description="Run BOTH", button_style="danger", layout=widgets.Layout(width='100%'))
cancel_button = widgets.Button(description="Cancel", button_style="info", layout=widgets.Layout(width='100%'))

output_box = widgets.Output()
progress_cpu = widgets.IntProgress(min=0, max=100, value=0, description='CPU Progress', bar_style='info', layout=widgets.Layout(width='100%'))
progress_ram = widgets.IntProgress(min=0, max=100, value=0, description='RAM Progress', bar_style='info', layout=widgets.Layout(width='100%'))

def set_buttons_state(running):
    cpu_button.disabled = running
    ram_button.disabled = running
    both_button.disabled = running
    cancel_button.disabled = not running

def run_in_thread(target):
    thread = threading.Thread(target=target)
    thread.start()

def run_cpu():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    cpu_stress(cpu_input.value, dur_input.value, output_box, progress_cpu)
    stress_running.clear()
    set_buttons_state(False)

def run_ram():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    ram_stress(ram_input.value, dur_input.value, output_box, progress_ram)
    stress_running.clear()
    set_buttons_state(False)

def run_both():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    combined_stress(cpu_input.value, ram_input.value, dur_input.value, output_box, progress_cpu, progress_ram)
    stress_running.clear()
    set_buttons_state(False)

def on_cancel_clicked(b):
    cancel_event.set()
    with output_box:
        output_box.append_stdout("Cancellation requested.\n")

cpu_button.on_click(lambda b: run_in_thread(run_cpu))
ram_button.on_click(lambda b: run_in_thread(run_ram))
both_button.on_click(lambda b: run_in_thread(run_both))
cancel_button.on_click(on_cancel_clicked)

cpu_row = widgets.HBox([widgets.Label("CPU cores:", layout=widgets.Layout(width='80px')), cpu_input, cpu_button], layout=widgets.Layout(gap='10px'))
ram_row = widgets.HBox([widgets.Label("RAM (MB):", layout=widgets.Layout(width='80px')), ram_input, ram_button], layout=widgets.Layout(gap='10px'))

display(warning, dur_input, cpu_row, ram_row, both_button, cancel_button, progress_cpu, progress_ram, output_box)


HTML(value="<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—setting…

BoundedIntText(value=30, description='Duration (s):', layout=Layout(width='150px'), max=600, min=5, step=5)

Button(button_style='danger', description='Run BOTH', layout=Layout(width='100%'), style=ButtonStyle())

Button(button_style='info', description='Cancel', layout=Layout(width='100%'), style=ButtonStyle())

IntProgress(value=0, bar_style='info', description='CPU Progress', layout=Layout(width='100%'))

IntProgress(value=0, bar_style='info', description='RAM Progress', layout=Layout(width='100%'))

Output()

[2025-06-02 05:46:54.046135] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]
[2025-06-02 05:46:54.046443+00:00] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]
[2025-06-02 05:46:54.046477+00:00] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]
[2025-06-02 05:46:54.046499+00:00] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]
[2025-06-02 05:46:54.046525+00:00] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]
[2025-06-02 05:46:54.046544+00:00] SUCCESS in cell (import multiprocessing
import threading
import time
import i...) [0.19s]


In [30]:
tracker.end_experiment()

[Experiment Tracking] Experiment exp-8b990987-20250602054649 completed and logged.
[Experiment Tracking] Detected Scaphandre metrics: ['scaph_context_switches_total', 'scaph_domain_energy_microjoules', 'scaph_domain_power_microwatts', 'scaph_forks_since_boot_total', 'scaph_host_cpu_frequency', 'scaph_host_disk_available_bytes', 'scaph_host_disk_total_bytes', 'scaph_host_energy_microjoules', 'scaph_host_load_avg_fifteen', 'scaph_host_load_avg_five', 'scaph_host_load_avg_one', 'scaph_host_memory_available_bytes', 'scaph_host_memory_free_bytes', 'scaph_host_memory_total_bytes', 'scaph_host_power_microwatts', 'scaph_host_swap_free_bytes', 'scaph_host_swap_total_bytes', 'scaph_process_cpu_usage_percentage', 'scaph_process_disk_read_bytes', 'scaph_process_disk_total_read_bytes', 'scaph_process_disk_total_write_bytes', 'scaph_process_disk_write_bytes', 'scaph_process_memory_bytes', 'scaph_process_memory_virtual_bytes', 'scaph_process_power_consumption_microwatts', 'scaph_processes_blocked_cur

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib

[Experiment Tracking] Exporting scaph_process_memory_bytes
[Experiment Tracking] Exporting scaph_process_memory_virtual_bytes
[Experiment Tracking] Exporting scaph_process_power_consumption_microwatts
[Experiment Tracking] Exporting scaph_processes_blocked_current
[Experiment Tracking] Exporting scaph_processes_running_current
[Experiment Tracking] Exporting scaph_self_cpu_usage_percent
[Experiment Tracking] Exporting scaph_self_domain_records_nb
[Experiment Tracking] Exporting scaph_self_memory_bytes
[Experiment Tracking] Exporting scaph_self_memory_virtual_bytes
[Experiment Tracking] Exporting scaph_self_socket_records_nb
[Experiment Tracking] Exporting scaph_self_socket_stats_nb
[Experiment Tracking] Exporting scaph_self_topo_procs_nb
[Experiment Tracking] Exporting scaph_self_topo_records_nb
[Experiment Tracking] Exporting scaph_self_topo_stats_nb
[Experiment Tracking] Exporting scaph_self_version
[Experiment Tracking] Exporting scaph_socket_energy_microjoules
[Experiment Tracking]

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mc-a4.lab.uvalight.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib